## Data

In [2]:
from _libs.pre_processing.pipeline import make_pipeline
from _libs.pre_processing.training_data import get_training_data, read_data
from _libs.paths import PATHS

In [3]:
X, y = read_data(PATHS["KIDNEY_DISEASE"], "class")

In [4]:
X.shape

(400, 24)

In [5]:
pipeline = make_pipeline(X)
pipeline.set_output(transform="pandas")

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'bp', 'bgr', 'bu',
                                                   'sc', 'sod', 'pot', 'hemo',
                                                   'pcv', 'wbcc', 'rbcc']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sg', 'al', 'su', 'rbc',
                                                   'pc', 'pcc', 'ba', 'htn',
                                                   'dm', 'cad', 'appet', 'pe',
                                                   'ane'])]))])

In [6]:
X_train, X_test, y_train, y_test = get_training_data(X, y, pipeline)

In [12]:
X_train

,num__age,num__bp,num__bgr,num__bu,num__sc,num__sod,num__pot,num__hemo,num__pcv,num__wbcc,...,cat__cad_yes,cat__appet_?,cat__appet_good,cat__appet_poor,cat__pe_?,cat__pe_no,cat__pe_yes,cat__ane_?,cat__ane_no,cat__ane_yes
212,-0.677336,-0.480635,1.405339,1.880664,1.573407,-0.601425,0.345408,-0.599548,-0.968506,0.156297,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
295,-0.441400,-0.480635,-0.562820,-0.658735,-0.387215,1.356638,-0.364755,0.911821,1.365392,0.473749,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
364,1.269138,0.262338,-0.402154,-0.272746,-0.422862,-0.057519,-0.400263,0.838095,0.751208,0.354704,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
251,-1.680065,0.262338,-1.044818,-0.435268,-0.369391,1.356638,-0.009674,1.649073,1.611066,0.553112,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
390,0.030473,0.262338,-0.656542,-0.658735,-0.405039,-0.275081,-0.329247,0.911821,1.611066,-0.835742,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,-0.323432,-1.223607,0.200344,0.702383,0.040557,0.377606,-0.222723,-1.005037,-1.337016,2.457826,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
106,-0.087496,1.005311,-0.790430,1.230578,0.539625,-1.145331,-0.080690,-2.405817,-2.688221,-0.756379,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
270,-1.680065,0.262338,-0.495876,-0.475898,-0.351567,0.812731,-0.222723,0.653782,0.259861,-0.478608,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
348,-0.795304,0.262338,-0.656542,-0.780626,-0.458510,1.030294,-0.400263,0.395744,0.628372,-0.438926,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## TPOT

In [14]:
from tpot import TPOTClassifier

automl = TPOTClassifier(generations=5, 
                        population_size=20, 
                        random_state=42, 
                        max_time_mins=1, 
                        n_jobs=1)

In [15]:
automl.fit(X_train, y_train)

c:\Users\Alvaro Lima\miniconda3\envs\tpot\lib\site-packages\tpot\tpot_estimator\estimator.py:456: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
c:\Users\Alvaro Lima\miniconda3\envs\tpot\lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49187 instead
  warnings.warn(
c:\Users\Alvaro Lima\miniconda3\envs\tpot\lib\site-packages\tpot\tpot_estimator\estimator.py:535: UserWarning: Labels are not encoded as ints from 0 to N. For compatibility with some classifiers such as sklearn, TPOT has encoded y with the sklearn LabelEncoder. When using pipelines outside the main TPOT estimator class, you can encode the labels with est.label_encoder_
  warnings.warn("Labels are not encoded as ints from 0 to N. For compa

TPOTClassifier(max_time_mins=1, random_state=42,
               search_space=<tpot.search_spaces.pipelines.sequential.SequentialPipeline object at 0x0000025921311B10>)

In [19]:
automl.fitted_pipeline_

Pipeline(steps=[('normalizer', Normalizer()),
                ('variancethreshold',
                 VarianceThreshold(threshold=0.0031917008133)),
                ('featureunion-1',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer()),
                                                ('passthrough',
                                                 Passthrough())])),
                ('featureunion-2',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer()),
                                                ('passthrough',
                                                 Passthrough())])),
                ('logisticregression',
                 LogisticRegression(C=27020.688345605467,
                                    l1_ratio=0.0468523318087, max_iter=1000,
                                    n_jobs=1, penalty='elasticnet',
                                    random_state=42, solver='saga'))])

In [31]:
from sklearn.metrics import classification_report

y_pred = automl.predict(X_test)

In [34]:
y_pred_processed = ["ckd" if i == 0 else "notckd" for i in y_pred]
print(classification_report(y_test, y_pred_processed))

              precision    recall  f1-score   support

         ckd       1.00      0.99      1.00       205
      notckd       0.98      1.00      0.99       115

    accuracy                           0.99       320
   macro avg       0.99      1.00      0.99       320
weighted avg       0.99      0.99      0.99       320

